# Projet de Data Mining (EPSI Rennes) - Application de la Méthodologie CRISP-DM
## (Version Finale Corrigée et Structurée)

### Phase 1-3 : Business & Data Understanding, Data Preparation

In [ ]:
# =============================================================================
# 1. IMPORT DES LIBRAIRIES
# =============================================================================
import pandas as pd
import numpy as np
import joblib
from wordcloud import WordCloud
import nltk
from nltk.corpus import stopwords
import re
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
pd.set_option('display.max_columns', None)

### Étape 1 : Préparation Complète des Données (Nettoyage + Feature Engineering)
Ce bloc exécute toutes les étapes de préparation en une seule fois pour garantir la présence de toutes les colonnes.

In [ ]:
# --- 1.1 Chargement et Nettoyage ---
df = pd.read_csv('fr-esr-insertion_professionnelle-master.csv', sep=';')

columns_mapping = {
    'annee': 'annee_diplome', 'discipline': 'discipline',
    'academie': 'academie', 'taux_dinsertion': 'taux_insertion',
    'emplois_stables': 'taux_emploi_stable',
    'salaire_net_median_des_emplois_a_temps_plein': 'salaire_median',
    'emplois_cadre': 'taux_emploi_cadre', 'femmes': 'part_femmes'
}
df_cleaned = df[columns_mapping.keys()].copy()
df_cleaned.rename(columns=columns_mapping, inplace=True)

cols_to_numeric = df_cleaned.columns.drop(['discipline', 'academie'])
for col in cols_to_numeric:
    df_cleaned[col] = pd.to_numeric(df_cleaned[col], errors='coerce')

df_cleaned.dropna(subset=['taux_insertion', 'salaire_median', 'academie'], inplace=True)

for col in ['taux_emploi_stable', 'taux_emploi_cadre', 'part_femmes']:
    group_median = df_cleaned.groupby('discipline')[col].transform('median')
    df_cleaned[col].fillna(group_median, inplace=True)
    df_cleaned[col].fillna(df_cleaned[col].median(), inplace=True)

df_cleaned.drop_duplicates(inplace=True)

# --- 1.2 Feature Engineering (Création des nouvelles colonnes) ---

# Feature 1 : Région administrative
region_mapping = {
    'Aix-Marseille': "Provence-Alpes-Côte d'Azur", 'Amiens': 'Hauts-de-France', 'Besançon': 'Bourgogne-Franche-Comté',
    'Bordeaux': 'Nouvelle-Aquitaine', 'Clermont-Ferrand': 'Auvergne-Rhône-Alpes', 'Corse': 'Corse',
    'Créteil': 'Île-de-France', 'Dijon': 'Bourgogne-Franche-Comté', 'Grenoble': 'Auvergne-Rhône-Alpes',
    'Guadeloupe': 'Guadeloupe', 'Guyane': 'Guyane', 'Lille': 'Hauts-de-France', 'Limoges': 'Nouvelle-Aquitaine',
    'Lyon': 'Auvergne-Rhône-Alpes', 'Martinique': 'Martinique', 'Mayotte': 'Mayotte', 'Montpellier': 'Occitanie',
    'Nancy-Metz': 'Grand Est', 'Nantes': 'Pays de la Loire', 'Nice': "Provence-Alpes-Côte d'Azur",
    'Normandie': 'Normandie', 'Orléans-Tours': 'Centre-Val de Loire', 'Paris': 'Île-de-France',
    'Poitiers': 'Nouvelle-Aquitaine', 'Reims': 'Grand Est', 'Rennes': 'Bretagne', 'La Réunion': 'La Réunion',
    'Strasbourg': 'Grand Est', 'Toulouse': 'Occitanie', 'Versailles': 'Île-de-France'
}
df_cleaned['region'] = df_cleaned['academie'].map(region_mapping)
df_cleaned.dropna(subset=['region'], inplace=True)

# Feature 2 : Grand Domaine
conditions = [
    df_cleaned['discipline'].str.contains('Droit|Gestion|Economie|AES|Commerce', case=False, na=False),
    df_cleaned['discipline'].str.contains('Lettre|Langue|Art|Communication|Traduction|Journalisme', case=False, na=False),
    df_cleaned['discipline'].str.contains('Humaine|Sociale|Histoire|Géographie|Socio|Psycho|Philo', case=False, na=False),
    df_cleaned['discipline'].str.contains('Enseignement|MEEF', case=False, na=False),
    df_cleaned['discipline'].str.contains('Science|Technologie|Santé|Ingénierie|Informatique|Math|Biologie|Physique|Chimie|STAPS', case=False, na=False)
]
choices = ['Droit, Éco & Gestion', 'Arts, Lettres & Langues', 'Sciences Humaines & Sociales', 'Enseignement', 'Sciences & Tech']
df_cleaned['grand_domaine'] = np.select(conditions, choices, default='Autres disciplines')

# Feature 3 : Indice d'Attractivité
salaire_norm = (df_cleaned['salaire_median'] - df_cleaned['salaire_median'].min()) / (df_cleaned['salaire_median'].max() - df_cleaned['salaire_median'].min())
insertion_norm = (df_cleaned['taux_insertion'] - df_cleaned['taux_insertion'].min()) / (df_cleaned['taux_insertion'].max() - df_cleaned['taux_insertion'].min())
df_cleaned['indice_attractivite'] = (0.6 * salaire_norm + 0.4 * insertion_norm) * 100

# Feature 4 : Qualité de l'Emploi
conditions_qualite = [
    (df_cleaned['taux_emploi_stable'] >= 80) & (df_cleaned['taux_emploi_cadre'] >= 80),
    (df_cleaned['taux_emploi_stable'] >= 65) & (df_cleaned['taux_emploi_cadre'] >= 65),
]
choices_qualite = ['Excellente', 'Élevée']
df_cleaned['qualite_emploi'] = np.select(conditions_qualite, choices_qualite, default='Standard')

# Feature 5 : Parité
df_cleaned['parite'] = pd.cut(df_cleaned['part_femmes'], bins=[0, 40, 60, 101], labels=['Majorité Masculine', 'Mixte', 'Majorité Féminine'], right=False)

print("Préparation et Feature Engineering terminés. Dimensions finales :", df_cleaned.shape)
print("\nColonnes présentes dans le DataFrame :", df_cleaned.columns.tolist())

### Étape 2 : Modélisation (Clustering et Régression)

In [ ]:
# --- Clustering K-Means ---
cluster_data = df_cleaned.groupby('academie').agg(
    salaire_median=('salaire_median', 'mean'),
    taux_insertion=('taux_insertion', 'mean'),
    taux_emploi_stable=('taux_emploi_stable', 'mean'),
    taux_emploi_cadre=('taux_emploi_cadre', 'mean')
).reset_index()

scaler = StandardScaler()
cluster_features = ['salaire_median', 'taux_insertion', 'taux_emploi_stable', 'taux_emploi_cadre']
cluster_data_scaled = scaler.fit_transform(cluster_data[cluster_features])

optimal_k = 4
kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
cluster_data['cluster'] = kmeans.fit_predict(cluster_data_scaled)

# On fusionne les résultats du clustering dans notre dataframe principal
df_final = df_cleaned.merge(cluster_data[['academie', 'cluster']], on='academie', how='left')
df_final['cluster'] = df_final['cluster'].astype('category')

# --- Régression RandomForest ---
features = ['grand_domaine', 'region']
target = 'salaire_median'
X = df_final[features]
y = df_final[target]

preprocessor = ColumnTransformer(transformers=[('cat', OneHotEncoder(handle_unknown='ignore'), features)])
model_regression = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1))
])
model_regression.fit(X, y)

print("\nModèles de Clustering et Régression entraînés.")

### Étape 3 : Export des Artefacts (Données, Modèles, WordClouds)

In [ ]:
# --- Text Mining ---
article_1_text = "L’insertion professionnelle des jeunes diplômés à un niveau record. La conjoncture a été particulièrement favorable aux sortants de l’enseignement supérieur en 2022. Le taux d’emploi des diplômés de master atteint 93 %, un an après la fin de leurs études, et ce, malgré l’inflation. Les salaires aussi sont en hausse. Le secteur des services et de l'industrie tirent la croissance."
article_2_text = "Insertion des jeunes diplômés 2024 : la confirmation des difficultés des jeunes femmes. Malgré un marché de l’emploi cadre dynamique, les jeunes femmes connaissent toujours des conditions d’insertion moins favorables que les hommes. Elles accèdent moins souvent au statut cadre et au CDI. L'écart de salaire persiste, notamment dans les fonctions informatiques et de la R&D. Les compétences techniques sont valorisées."

def process_text_for_wordcloud(text, custom_stopwords=[]):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    tokens = text.split()
    try: stop_words = set(stopwords.words('french'))
    except LookupError: nltk.download('stopwords')
    stop_words = set(stopwords.words('french'))
    stop_words.update(custom_stopwords)
    cleaned_tokens = [word for word in tokens if word not in stop_words]
    return " ".join(cleaned_tokens)

custom_stopwords = ['cest', 'dun', 'dune', 'plus', 'apec', 'cette']
cleaned_text_1 = process_text_for_wordcloud(article_1_text, custom_stopwords)
cleaned_text_2 = process_text_for_wordcloud(article_2_text, custom_stopwords)
wordcloud_1 = WordCloud(width=800, height=400, background_color='white', colormap='viridis').generate(cleaned_text_1)
wordcloud_2 = WordCloud(width=800, height=400, background_color='white', colormap='plasma').generate(cleaned_text_2)
wordcloud_1.to_file("wordcloud_article1.png")
wordcloud_2.to_file("wordcloud_article2.png")

# --- Sauvegarde des livrables ---
df_final.to_csv('insertion_pro_master_final_v2.csv', index=False)
joblib.dump(model_regression, 'salary_predictor_model.joblib')

print("\nExport terminé : 'insertion_pro_master_final_v2.csv', 'salary_predictor_model.joblib', et les WordClouds sont prêts.")